## Introduction
 
:::{.notes}
on s'est intéressé au marché de la voiture et on s'est posé la question de ce qu'on peut prédire sur les voitures. Quand on pense à ça, on pense forcément au prix, comment prédire le prix d'un véhivule?
 
La plupart des français utilisent la côte argus mais est-ce que vous vous êtes déjà demandé comment celle-ci fonctionne?
 
- Argus :
Elle est basée sur le cours-moyen d'argus qui est constitué à l'aide de professionnels du secteur, d'un concessionnaire allant jusqu'au constructeur, mais aussi à l'aide de simples particuliers et des annonces de ventes de véhicule.
 
 
En apprenant cela, nous avons eu comme objectif de créer notre propre estimateur de prix de voiture.
 
Pour ce faire, vu qu'on est que 2 jeunes étudiants et ne sommes pas entourés de professionnels, la seule option que nus avons est de scraper des annonces de voitures d'occasion en ligne.
 
Avant de vous montrer comment on a scrapper tout ça on va directement passer au + croustillant qui est une démonstration de l'application que nous avons créée.
 
Présentation Streamlit:
 
on se retrouvr sur l'accueil de notre app, qui est une app streamlit. On va vous expliquer plus tard ce que c'est.
Sur notre app, nous avons 3 onglets : accueil, acheteur et vendeur.
 
Démo acheteur, démo vendeur.
On va pouvoir visualiser
 
démo acheteur : Hassan est-ce que tu es interessé par l'achat d'un véhicule?
 
 
si lien inexistant : c'était tellement une bonne affaire qu'elle est déjà partie...
 
démo vendeur : toutes les données présentes dans l'onglet acheteur ont servi à entrainer les modèles de prédiction.
 
on a une sorte de transparence sur comment est calculée la côte.
 
Est-ce que vous avez une voiture à vendre? ou des caractéristiques
:::
 
 
 
*CarScraping qu'est-ce que c'est ?*
 
- Comment se calcule la **Côte ARGUS** ?
 
> Elle est basée sur le cours-moyen d'argus qui est constitué à l'aide de professionnels du secteur, d'un concessionnaire allant jusqu'au constructeur, mais aussi à l'aide de simples particuliers et des annonces de ventes de véhicule.
 
- Inconvénient : Payant, manque de transparence.
 
- **Objectif** : créer notre propre côte ARGUS gratuite et sans filtre  !
 
 
 
## Web Scraping 🕸️
 
:::{.notes}
Alors maintenant qu'on a vu le produit final, on va le disséquer pour voir comment on en est arrivé là avec une approche technique!
 
Vous avez vu le voir peut être mais il y a des liens dans l'onglet acheteur qui renvoie vers le site "La Centrale".
 
Alors nous avons scrapé ce site et l'intégralité des 300 000 annonces qui étaient présentes sur le site au moment du scraping.
 
Pour la partie scraping
 
 
On va vous montrer le processus de scraping en 3 parties : récupération des noms des marques, puis la récupération des annonces, et enfin la récupération des caractéristiques.
 
pas d'utilisation de Selenium car le site bloque tous les bot.
 
:::
 
> C'est une technique d'extraction de données présentes sur des sites  web par l'utilisation d'un script ou programme informatique.
 
Données récupérées depuis **La Centrale** :
 
- \+ de **300 000** annonces
- \+ de **150** marques
- \+ de **900** modèles
 
 
Extraction en 3 parties :
  - Récupération des noms des marques
  - Récupération des annonces
  - Récupération des caractéristiques
 
 
 
## Processus du Scraping : 1/3
 
:::{.notes}
 
On a un module `webscraping.py` qui nous permet d'extraire toutes ces données depuis le site.
 
 
Il faut savoir que le nombre maximum de pages sur La Centrale est de 500.
 
Avec 16 annonces par page, on n'atteint pas les 300k annonces annoncés dans l'accueil du site.
 
Pour y parvenir, nous avons décidé de scraper 500 pages d'annonces par marque et par modèles de véhicule à l'aide de requêtes http.
 
Pour obtenir ces différentes marques, il suffit juste d'aller dans l'accueil du site, de cliquer sur la listbox "Modèle" et d'ensuite télécharger la page.
 
 
 
:::
 
- Utilisation des 2 packages python : `request` et `bs4`.
 
- Récupération de la liste des marques et des modèles associés.
  
 
- Transformer en format JSON.
 
<div style="max-height: 100px; overflow-y: scroll;">
 
:::{.fragment}

```{json}
{
    "AC": [
        "aceca",
        "cobra"
    ],
    "AIWAYS": [
        "u5"
    ],
}
```

 
:::

## Processus du Scraping : (2/3)
:::{.notes}

Une fois qu'on a récupéré les marques, on peut maintenant à l'aide d'une boucle et de fonctions bien ficellées, extraire toutes les annonces sur le site. 
*extrait f° recup_pages()*

On va récupérer les annonces directement depuis l'url pour pouvoir pallier le problème des 500 pages maximum.

Cette fonction : `extraire_toutes_annonces()`, en contient d'autre. Ce qui permet d'iterer sur chaque marques et modeles disponible, grace au fichier JSON crée préalablement. Ensuite on effectue une requete http a l'aide de requetes (on montre l'exemple de la fonction recup_page).

:::

- Extraction de toutes les annonces disponibles sur le site à l'aide de la fonction `extract_toutes_annonces()`, elle permet :
  - d'iterer sur chaque marques et modèles de véhicule
  - d'extraire les pages correspondantes

```py
def recup_page(numero_page: int, marque: str, modele: str) -> BeautifulSoup:
    adresse = f"https://www.lacentrale.fr/listing?makesModelsCommercialNames={marque.upper()}%3A{modele.upper()}&options=&page={numero_page}"
    ...
    requete = rq.get(url=adresse)
    page = BeautifulSoup(requete.content, "html.parser")
``` 
  - d'avoir une sortie sous forme de liste d'annonces
  
```py
def recup_annonces(page: BeautifulSoup) -> list:
    annonces = page.find_all('div', class_='searchCardContainer')
    return list(annonces)
``` 


## Processus du Scraping : (3/3)

:::{.notes}

Une fois toutes les pages récuperées, sous cette forme de liste **pointe la slide**, on récupère chacune des caractéristiques du v"hicule pour chq annonce. 
La fonction permettant de récuperer toutes les caractéristiques d'une annonce s'appelle recup_info_voitures() qui elle aussi fait appel à pls fonctions dédiées à pls caractéristiques. Ces fonctions permette à l'aide de bs4, de recuperer des information précise stocké dans certain élement de la page. 

Chacune des caractéristiques des voitures est stockée dans une dataclass `voiture`. 
:::

![](imgs/sorties_annonces.png){fig-align="center" width="0.5"}

- Extraction des caractéristiques des véhicules.

```py
def recup_informations_voiture(annonce: str) -> voiture:
    marque = recup_nom_vehicule(annonce)
    cylindre = recup_cylindre(annonce)
    annee, kilometrage, boite, energie = recup_caracteristiques(annonce)
    prix = recup_prix(annonce)
    position_marché = recup_position_marché(annonce)
    garantie = recup_garantie(annonce)
    lien = recup_href(annonce)
    return voiture(marque=marque, cylindre=cylindre, annee=annee, kilometrage=kilometrage, boite=boite, energie=energie, prix=prix, position_marché=position_marché, garantie=garantie, lien=lien)

def recup_nom_vehicule(annonce: str) -> str: 
    nom_vehicule = annonce.find_all('h2', class_='Text_Text_text Vehiculecard_Vehiculecard_title Text_Text_subtitle2')
    return nom_vehicule[0].text
```

----


:::{.notes}
Et une fois le tout executé, les données sont exportés sous format brut JSON. 

le nb d'annonces étant très conséquent et l'extraction étant très longue, nous avons fait l'extraction en 4 parties. Nous avons obtenus donc  'fichiers json qu'on a fusionner en un fichier à l'aide de la f° `fusionner_fichiers_json()`. 
:::

✅ Résultats 4 fichiers JSON obtenu :


```
json/
  {} data_a_d.json
  {} data_e_l.json
  {} data_m_p.json
  {} data_r_z.json
```

Un extrait du fichier JSON obtenu : 

```{json}
[
    {
        "marque": "CITROEN C3 III",
        "cylindre": "1.2 PURETECH 110 FEEL",
        "annee": "2019",
        "kilometrage": "10 698 km",
        "boite": "Automatique",
        "energie": "Essence",
        "prix": "15 990 €",
        "position_marché": "Bonne affaire",
        "garantie": "Garantie 12 mois",
        "lien": "https://www.lacentrale.fr/auto-occasion-annonce-69112858137.html"
    },
    ...
]

```


On les fusionne à l'aide de la fonction `fusionner_fichiers_json` pour obtenir un seul fichier contenant toutes les données extraites.



## 🧼 Data cleaning 🧹

:::{.notes}

Une fois notre fichier json obtenue, chose inevitable nous avons des données mal renseignées, des pb de types de données mais aussi des informations à extraire de ces données brutes.

Pour récupérer le + d'informations possibles sur ces données, nous avons décidé d'utiliser la librairie polars, ce qui rend le tout très lisible et très efficace.

:::

- Essentiel pour pouvoir exploiter les données.
- Permet de récupérer + d'informations.


Pour le nettoyage de la base de données, nous avons utilisé le package `Polars` 🐻‍❄️, qui possède de nombreux avantages comme :  

-   la création de pipelines 🛢️
-   un code plus lisible 👀
-   dans certains cas, + efficace que `Pandas` 🐼

C'est le module `datacleaning.py` qui contient toutes les fonctions essentiels pour résoudre certains problèmes.

## Les problèmes rencontrées ⚠️

:::{.notes}

Dans cette partie, nous allons vous présenter rapidement les pb qu'on a rncontrées, et les fonctions qui ont servi à les règler et qui nous ont permis de nettoyer la base de données et d'avoir une BDD exploitable.


- modele_marque_generation

pb rencontré, nom de la f°, et la sortie.  (faire figurer l'ex des docstring)

:::

- Séparer la marque, le modèle et la génération à l'aide de `get_marque_modele_generation()`

$\rightarrow$ [`'CITROEN C3 III'`]{.fragment .semi-fade-out} [->]{.fragment} [`('CITROEN', 'C3', 'III')`]{.fragment}

- Corriger le type des variables `kilometre`, `prix` et `annee` avec `get_km_prix_annee` : 

$\rightarrow$ [`"100 000 km", "15 000 €", "2015"`]{.fragment .semi-fade-out} [->]{.fragment} [`(100000, 15000, 2015)`]{.fragment}

- Diviser la chaine de caractere de `cylindre` en plusieurs variable avec `get_cylindre`: 

$\rightarrow$ [`"6.3 V8 460 GT"`]{.fragment .semi-fade-out} [->]{.fragment} [`('6.3', 'V8', 460, 'GT')`]{.fragment}

## Le gazoduc 🛢️

:::{.notes}
f° qui permet de faire tout le traitement automatiquement à l'aide des autres fonctions. 

Comme on peut le voir, il y a d'autre fonction dans cette pipeline que l'on ne vous a pas présenté. On va donc le faire très rapidement. 

get_garantie traite les données de la colonne "garantie" dans en transformant le Garantie 12 mois en gardant uniquement les valeurs numériques. Elle contient aussi d'autre filtre pour écarter les valeurs abérantes 

filter_data permet de supprimer les valeurs abérante pour le kilométrage et l'année 2024. 

supp_doublons permet de supprimer les annonces en double en ce basant sur les liens des annonces. 

supp_na: Elle permet aussi de supprimer les valeurs manquantes pour la puissance. et pour les marques et modeles de véhicule

Une fois le dataframe nettoyé il sera exporté en format parquet car parquet a une Compression efficace et permet d'avoir des économies de stockage. 
:::


C'est une fonction qui applique un pipeline de traitement de données sur le DataFrame donné.
 
```py
def gazoduc(data: pl.DataFrame, nom_marques_modeles: pl.DataFrame) -> pl.DataFrame:
    data = (data.pipe(get_marque_modele_generation, nom_marques_modeles)🛢️
                .pipe(get_km_prix_annee)                                🛢️
                .pipe(get_garantie)                                     🛢️
                .pipe(get_cylindre)                                     🛢️
                .pipe(filter_data)                                      🛢️
                .pipe(supp_doublons)                                    🛢️  
                .pipe(supp_na)                                          🛢️
        )
    return data
```

<div style="overflow-x: auto;">
  <table style="max-width: 800px; font-size: 18px;">
    <caption>Extrait du dataframe</caption>
    <thead>
      <tr>
        <th>année</th>
        <th>kilométrage</th>
        <th>boîte</th>
        <th>énergie</th>
        <th>prix</th>
        <th>position marché</th>
        <th>garantie</th>
        <th>lien</th>
        <th>marque</th>
        <th>modèle</th>
        <th>génération</th>
        <th>cylindre</th>
        <th>moteur</th>
        <th>puissance</th>
        <th>finition</th>
        <th>batterie</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>2020</td>
        <td>107410</td>
        <td>Manuelle</td>
        <td>Essence</td>
        <td>13050</td>
        <td>Offre équitable</td>
        <td>12</td>
        <td><a href="https://www.lacentrale.fr/auto-occasion-annonc...">Lien vers l'annonce</a></td>
        <td>CITROEN</td>
        <td>C3</td>
        <td>III</td>
        <td>1.2</td>
        <td>PURETECH</td>
        <td>82</td>
        <td>S&S FEEL BUSINESS</td>
        <td>None</td>
        <td>-</td>
      </tr>
    </tbody>
  </table>
</div>



Ce dataframe sera exporté en format `.parquet` qui permet de stocker des bases volumineuse à moindre coût. 

## 📖 Machine learning 📖


:::{.notes}
- split
- les modèles -> pourquoi avoir choisi ces modèles ? car après test, ces modèles se sont averée être meilleurs.
- les grilles de paramètres associées -> pourquoi ces grilles ? car après test, se sont les résultats qui sont sortie le plus souvent.
- trouver le meilleur modèle (CV) -> on montre la fonction 
- export du Modèle et préprocesseur
- import du modèle et utilisation pour prédiction

:::

Pour des raisons d'efficacité pour la prédiction, chaque marque de véhicule aura son propre modèle de prédiction.

Ces modèles de prédiction ne peuvent être choisi au hasard. Il faut effectuer des tests et choisir le modèle le plus adapté.



- package utilisé : `sklearn`

L'utilisation de ce package à plusieurs avantages, tel que ça documentation et une utilisation avec des Pipelines possibles. Cependant, il ne fonctionne pas avec des bases de données `Polars`.


- split des données avec `split_data`
- preprocesseur avec `get_preprocessor`
- modèles et grille de parametres avec `set_models` et `get_params`
- pour récupérer et exporter les meilleurs modèle : `get_all_models`
- `predict_prix`

## Le split 👨‍👩‍👧‍👦


Données d'entrainement: échantillon de 80% d'observations de la marque

Données de test: les 20% restants.

Le split est effectué à l'aide de `split_data` : 

```py
def split_data(data: pl.DataFrame, marque: str) -> tuple[pl.DataFrame, pl.DataFrame, pd.DataFrame, pd.DataFrame, np.ndarray, np.ndarray]:

    X = data.filter(pl.col("marque") == marque)
    y = X.select('prix')
    X = X.select(pl.exclude("position_marché")
                 ).select(pl.exclude("lien")
                          ).select(pl.exclude("garantie")
                                   ).select(pl.exclude("prix"))

    # Split des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X.to_pandas(), y.to_numpy(), test_size=0.2, random_state=21)
    return X, y, X_train, X_test, y_train, y_test

```

## Le preprocesseur ⏹️

La plupart des modèles ne supportent pas les variables qualitatives et requiert donc un prétraitement. 

`OneHotEncoder` est utilisé pour convertir les caractéristiques catégorielles en une représentation binaire.

Exemple : 

$\rightarrow$ [`'boite' = 'Automatique''`]{.fragment .semi-fade-out} [->]{.fragment} [`boite.Automatique = 1`]{.fragment}

`StandardScaler` est utilisé pour effectuer une mise à l'échelle des caractéristiques numériques. Il standardise les données en soustrayant la moyenne et en divisant par l'écart type, de sorte que chaque caractéristique ait une moyenne de zéro et une variance de un.

## Les modèles 📊

Après quelque test, nous avons décidé d'utiliser uniquement les modèles suivant : 

- Regression linéaire 📈
- K-neighbors 👬👭
- Random Forest 🌳

Pour les paramètres, plusieurs grilles ont été testé, voici les résultats intermédiaires qui nous ont permis de déterminer la grille finale.

## Le nombre de voisins 👬👬👬


In [ ]:
import json
import re

import plotly.express

def read_from_html(file):
    with open(file) as f:
        html = f.read()
    call_arg_str = re.findall(r'Plotly\.newPlot\((.*)\)', html[-2**16:])[0]
    call_args = json.loads(f'[{call_arg_str}]')
    plotly_json = {'data': call_args[1], 'layout': call_args[2]}    
    return plotly.io.from_json(json.dumps(plotly_json))
read_from_html("plots/KNeigbors_graph.html")

## Streamlit


:::{.notes}
expliquer streamlit
- onglet acheteur : fonctionnement avec des requêtes, 
- onglet vendeur : import du modèle et prédictions estimation du prix 



:::